<a href="https://colab.research.google.com/github/hjiang13/CODEBERT-REGRESSION/blob/main/CODEBERT-REGRESSION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/hjiang13/CODEBERT-REGRESSION.git

Cloning into 'CODEBERT-REGRESSION'...
remote: Enumerating objects: 312, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 312 (delta 48), reused 42 (delta 21), pack-reused 235
Receiving objects: 100% (312/312), 3.40 MiB | 12.56 MiB/s, done.
Resolving deltas: 100% (212/212), done.


In [2]:
!ls

sample_data


In [ ]:
!rm -r CodeBERT-classification/

In [4]:
import os
os.chdir('CODEBERT-REGRESSION/code')

In [ ]:
!git pull

Updating 88521ab..7294e2f
error: Your local changes to the following files would be overwritten by merge:
	code/train_resilience.sh
Please commit your changes or stash them before you merge.
Aborting


In [5]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from transformers import BertTokenizer, BertModel, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from torch import nn
import json

from transformers import (WEIGHTS_NAME, AdamW, get_linear_schedule_with_warmup,
                          RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)
from transformers import LongformerConfig, LongformerModel

In [6]:
trainDataPath = "../dataset/benign_train_resilience_r.jsonl"
evalDataPath = "../dataset/benign_test_resilience_r.jsonl"
train_data = pd.DataFrame( {"code": [], "label": []})
eval_data = pd.DataFrame( columns=['code', 'label'])


In [7]:
with open(trainDataPath, "r") as data_file:
    i = 0
    for line in data_file:
        line = json.loads(line)
        lineList= [[line["code"], line["label"]]]
        df_line = pd.DataFrame(lineList, columns=['code', 'label'])
        train_data = pd.concat([train_data, df_line])
        i += 1
        if i > 5:
            break

with open(evalDataPath, "r") as data_file:
    i = 0
    for line in data_file:
        line = json.loads(line)
        lineList= [[line["code"], line["label"]]]
        df_line = pd.DataFrame(lineList, columns=['code', 'label'])
        eval_data = pd.concat([eval_data, df_line])
        i += 1
        if i > 5:
            break

In [8]:
eval_data

,code,label
0,"#include ""npbparams.h""\n#include <stdlib.h>\n#...",0.614604
0,\n#include <iostream>\n#include <fstream>\n#in...,0.456389
0,#include <iostream>\n#include <fstream>\n#incl...,0.330629
0,#include <stdio.h>\n#include <stdlib.h>\n#incl...,0.793103
0,/*\n *****************************************...,0.649087
0,#include <cmath>\n#include <iostream>\n#includ...,0.789047


In [9]:
train_data_array = train_data['code'].to_numpy()
train_data_label_array = train_data['label'].to_numpy()

In [10]:
eval_data_array = eval_data['code'].to_numpy()
eval_data_label_array = eval_data['label'].to_numpy()

In [ ]:
train_data_array[0]

In [11]:
tokenizer = RobertaTokenizer.from_pretrained("neulab/codebert-cpp")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.54k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

In [12]:
tokenizer(train_data_array[0])

Token indices sequence length is longer than the specified maximum sequence length for this model (1239 > 512). Running this sequence through the model will result in indexing errors


{'input_ids': [0, 42326, 936, 445, 16, 11, 1205, 640, 1401, 4, 41040, 14385, 3153, 4, 175, 73, 10800, 10092, 73, 879, 31104, 12, 43892, 17265, 12, 246, 73, 25324, 33618, 73, 428, 14247, 12, 2670, 73, 41045, 50118, 10431, 47209, 28696, 32131, 73, 620, 34836, 42964, 4, 298, 15698, 50118, 50118, 49795, 50118, 1009, 1437, 1437, 1437, 2730, 35, 1437, 8376, 50118, 1009, 1437, 1437, 1437, 1412, 35, 1132, 4, 3546, 4, 3789, 753, 35, 3669, 35, 4419, 1437, 1437, 1437, 1437, 1437, 1437, 1437, 50118, 12606, 73, 50118, 10431, 47209, 28696, 32131, 73, 620, 34836, 42964, 4, 298, 15698, 50118, 50118, 10928, 49010, 49008, 131, 50118, 50118, 20836, 6979, 234, 5457, 316, 3079, 131, 50118, 50118, 24262, 579, 10975, 487, 46386, 487, 44082, 50118, 2544, 821, 10975, 487, 46386, 487, 44082, 50118, 50118, 2544, 1049, 43048, 25522, 50118, 18178, 12592, 46469, 49445, 46797, 1215, 42018, 73, 25158, 20178, 1297, 22, 338, 1297, 49008, 179, 4397, 50118, 1437, 6979, 326, 90, 131, 50118, 1437, 14194, 506, 46469, 207, 4

In [13]:
len(tokenizer(train_data_array[0])["input_ids"])

1239

In [14]:
windowLenght = 512
window_num = round(len(tokenizer(train_data_array[0])["input_ids"])/windowLenght)
window_num

2

In [25]:
i = 0
[[tokenizer(train_data_array[0])["input_ids"][i*windowLenght:(i+1)*windowLenght]]]

[[[0,
   42326,
   936,
   445,
   16,
   11,
   1205,
   640,
   1401,
   4,
   41040,
   14385,
   3153,
   4,
   175,
   73,
   10800,
   10092,
   73,
   879,
   31104,
   12,
   43892,
   17265,
   12,
   246,
   73,
   25324,
   33618,
   73,
   428,
   14247,
   12,
   2670,
   73,
   41045,
   50118,
   10431,
   47209,
   28696,
   32131,
   73,
   620,
   34836,
   42964,
   4,
   298,
   15698,
   50118,
   50118,
   49795,
   50118,
   1009,
   1437,
   1437,
   1437,
   2730,
   35,
   1437,
   8376,
   50118,
   1009,
   1437,
   1437,
   1437,
   1412,
   35,
   1132,
   4,
   3546,
   4,
   3789,
   753,
   35,
   3669,
   35,
   4419,
   1437,
   1437,
   1437,
   1437,
   1437,
   1437,
   1437,
   50118,
   12606,
   73,
   50118,
   10431,
   47209,
   28696,
   32131,
   73,
   620,
   34836,
   42964,
   4,
   298,
   15698,
   50118,
   50118,
   10928,
   49010,
   49008,
   131,
   50118,
   50118,
   20836,
   6979,
   234,
   5457,
   316,
   3079,
   131,
  

In [26]:
chunckPool= pd.DataFrame( {"code": [], "label": []})
chunckList = []
for i in range(window_num-1):
    chunckList.append([tokenizer(train_data_array[0])["input_ids"][i*windowLenght:(i+1)*windowLenght],train_data_label_array[0]])
chunckList.append([tokenizer(train_data_array[0])["input_ids"][-512:], train_data_label_array[0]])
chunckList
for i in range(window_num):
    df_chunckList = pd.DataFrame([chunckList[i]], columns=['code', 'label'])
    chunckPool = pd.concat([chunckPool, df_chunckList])

In [27]:
chunckPool

,code,label
0,"[0, 42326, 936, 445, 16, 11, 1205, 640, 1401, ...",0.426
0,"[1437, 1437, 114, 36, 1253, 45994, 321, 43, 25...",0.426
